In [1]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time

In [2]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=8)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/hjj/Desktop/GAN-SMF/train/'

test_path='/home/hjj/Desktop/GAN-SMF/test/'

global sample_num, rmat_num, series_num
sample_num=1 #output of G
rmat_num=28  #row nums of the matrix for the input of CNN 
series_num=3#input of D



In [4]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=round(np.float64(float(energy_string)),5)
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [ ]:
global extend_num, move_num

In [ ]:
extend_num=1000

In [5]:
move_num=get_energy(train_path+'00000/')
print(move_num)

-175.79261


In [ ]:
base_pxrd_s=mg.Structure.from_file('/home/hjj/Desktop/GAN-SMF/train/00000/CONTCAR')
base_pxrd=patt_xrd.get_pattern(base_pxrd_s)

In [ ]:
#randomly select the file path of the input structure
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    #pos_name=folder+'/POSCAR'
    #out_name=folder+'/OUTCAR'
    return folder

#transform 'POSCAR' into the formate of pymatgen
def tomgStructure(folder):
    POSfile=folder+'/CONTCAR'      
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS

###
##get the pxrd data of a structure via pymatgen, and screen these peaks by intensity and angle
### The fourth method

def get_xrdmat4(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)
    xrd_data4.y=xrd_data4.y-base_pxrd.y
    i_column = rmat_num
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if abs(xrd_data4.y[i])>0.00000000001:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.dot(100,xrd_y)
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4
###
##input_data_as_knowlegde
###
'''
def get_Gibbs(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    Gibbs=np.float64(float(energy_string))
    Gibbs=round(Gibbs,6)
    return Gibbs
'''
##
###calculate the number of atoms in a POSCAR via pymatgen

def get_atoms_num(folder2):
    xxx=tomgStructure(folder2)
    anum=len(xxx.sites)
    return anum


###
## generate the graph-structure matrix as the input of GAN
###
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat4(Random_Structure)
    multimat4_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat4_RS = np.asarray((np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat4_RS

In [ ]:
class GNet(nn.Module):
    
    def __init__(self, input_size=(sample_num,28,28)):
        super(GNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.Dlstm=nn.LSTM(
            input_size=series_num,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        )
        self.out=nn.Sequential(
            nn.Linear(32,10),
            nn.ReLU(),
            nn.Linear(10,1),
            
        )
        #nn.Linear(32,1)
        #nn.Relu
        #nn.Linear
        #nn.Sigmoid
        
    def forward(self,x):
        D_out,(h_n,h_c)=self.Dlstm(x,None)
        out = self.out(D_out[:,-1,:]) #(batch,time step,input)   
        return out

In [ ]:
mat_Gl=[]    
mat_Dl=[]
pre_d_real=[]
pre_d_fake=[]
error_test=[]
error_train=[]
r2=[]

In [ ]:
G1=GNet()
D1=DNet()

In [ ]:
opt_D1=torch.optim.Adam(D1.parameters(),lr=0.1)
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.052)

In [ ]:
#initialization of the series used as the input of D
train_series=[]
for i in range(series_num):
    path_s=random_xxpsk(train_path)
    ee1=get_energy(path_s)
    ee1=linear_transform(ee1)
    train_series.append(ee1)

In [ ]:
file_path=train_path
used_file_set=[]
for step in range(1,4):       
 
    #modules: control the POSCARs used for training
    sample_path=[]
    for i in range(1,sample_num + 1):
        path_ = random_xxpsk(file_path)
        sample_path.append(path_)
        used_file_set.append(path_)
   
    # update the train-series when x=real
    E_Gibbs=0
    for path1_ in sample_path:
        try:
            total_energy=get_energy(path1_)
            E_Gibbs=linear_transform(total_energy)
            
        except:
            print(path1_)
         
        train_series.pop(-1)
        train_series.append(E_Gibbs)
        
        
    input_series_D=np.asarray(train_series,dtype=np.float64)       
    input_series_D=Variable(torch.from_numpy(input_series_D[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    d_real=D1(input_series_D)
    pre_d_real.append(d_real.data.numpy().mean())
    
    # update the train-series when x=fake 
    g_in=[]
    for path2_ in sample_path:
        path2_=str(path2_)                
        
        try:
            tomgS=tomgStructure(path2_)            
            gin=GANs_Gmat(tomgS)  
        except:
            pass
        g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    G_fake=G1(g_in)
    Gout=round(G_fake.data.numpy().mean(),6)
    
    train_series.pop(0)
    train_series.append(Gout)
    
        
    input_series_D=np.asarray(train_series,dtype=np.float64)       
    input_series_D=Variable(torch.from_numpy(input_series_D[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    
    d_fake=D1(input_series_D)
    pre_d_fake.append(d_fake.data.numpy().mean())
    
    D1_loss=-torch.mean(torch.log(d_real)+torch.log(1.- d_fake))   
    dd=D1_loss.data.numpy().mean()
    mat_Dl.append(dd)
    
    G1_loss=torch.mean(d_fake)
    gg=G1_loss.data.numpy().mean()
    mat_Gl.append(gg)
    
    if step%1==0:
        opt_D1.zero_grad()
        D1_loss.backward(retain_graph=True)
        opt_D1.step()
        
        opt_G1.zero_grad()
        G1_loss.backward()
        opt_G1.step()
    else:
        opt_D1.zero_grad()
        D1_loss.backward()
        opt_D1.step()
    


    if step%1==0:
        print(step)
        print('error: ',abs(inverse_transform(Gout)-inverse_transform(E_Gibbs)))        
        print(dd)
        print(gg)
        print("real:",d_real.data.numpy().mean())
        print("fake:",d_fake.data.numpy().mean())
        
        
        
        
    
    
    if step%1==0:
        print('----------------------------------test-------------------------')
        file_path_test=test_path
        E_Gibbs_1=[]
        E_Gmodel_1=[]
        for step_test in range(8):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_energy(path1_)
                    tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            tt_energy=inverse_transform(float(tt_energy))
            print('DFT',tt_energy)
            E_Gibbs_1.append(tt_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
                    
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        
        #print(Gout.shape)
    
            G_data=round(inverse_transform(Gout.data.numpy().mean()),6)
            print('G_predict',G_data)
            E_Gmodel_1.append(G_data)

        
            
            
            print('error',abs(tt_energy-G_data))
            
        print('------------------end-test----------------------------')
        xxx=abs(abs(np.asarray(E_Gibbs_1))-abs(np.asarray(E_Gmodel_1))).mean()
        print(xxx)
        error_test.append(xxx)
        
        X=np.asarray(E_Gibbs_1)
        Y=np.asarray(E_Gmodel_1)

        xbar=X.mean()
        ybar=Y.mean()
        SSR=0
        varX=0
        varY=0
        for i in range(len(X)):
            diffxxbar=X[i]-xbar
            diffyybar=Y[i]-ybar
            SSR+=(diffxxbar*diffyybar)
            varX+=diffxxbar**2
            varY+=diffyybar**2
    
        SST=math.sqrt(varX+varY)
        R2=(SSR/SST)**2
        print("R2:",R2)
        r2.append(R2)
    else:
        pass
    


In [ ]:

E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
MSEset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
start=time.time()        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        #GGG=inverse_transform(GGG)
        E_Gibbs_test.append(GGG)
        
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        mse=(G_data-GGG)**2
        abserrset.append(abserr)
        MSEset.append(mse)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)
end=time.time()
print(end-start)

In [ ]:
print(np.asarray(abserrset).mean())

print(np.asarray(MSEset).mean())

print(np.sqrt(np.asarray(MSEset).mean()))

In [ ]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
tMSEset=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)

In [ ]:
print(testfile)

In [ ]:

#'''        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        
        #GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        tmse=(G_data-GGG)**2
        tMSEset.append(tmse)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [ ]:
print(np.asarray(abs_t_errset).mean())

print(np.asarray(tMSEset).mean())

In [ ]:
print(np.sqrt(np.asarray(tMSEset).mean()))

In [ ]:
torch.save(G1.state_dict(),"G1.pkl") 
torch.save(D1.state_dict(),"D1.pkl")

In [ ]:
'''
sample_path=[]
sample_path.append('/home/mii/Desktop/cpx222/train/00000')
g_in=[]
for path2_ in sample_path:
    path2_=str(path2_)                
        
    try:
        tomgS=tomgStructure(path2_)
            #print(tomgS)
        gin=GANs_Gmat(tomgS)
                
            #print(gin)
    except:
        pass
    g_in.append(gin)
       
g_in=np.asarray(g_in)
g_in=g_in[np.newaxis,:,:,:] 
g_in=np.asarray(g_in,dtype=np.float64) 
g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
Gout=G1(g_in)
        
        #print(Gout.shape)
    
G_data=round(inverse_transform(Gout.data.numpy().mean()),6)
print('G_predict',G_data)
'''